In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sklearn.datasets as sk_data
import sklearn.model_selection as sk_selection
import sklearn.metrics as sk_metrics
import sklearn.ensemble as sk_ensemble
import random_neural_net_models.tabular as rnnm_tab
import torch.optim as optim
import random_neural_net_models.losses as rnnm_loss
import random_neural_net_models.learner as rnnm_learner
import random_neural_net_models.data as rnnm_data
import random_neural_net_models.utils as rnnm_utils
from pathlib import Path
from torch.utils.data import DataLoader

In [ ]:
rnnm_utils.make_deterministic(42)

In [ ]:
device = rnnm_utils.get_device()
device

In [ ]:
X, y = sk_data.load_iris(return_X_y=True)

In [ ]:
X.shape

In [ ]:
X

In [ ]:
y

In [ ]:
import numpy as np

np.isfinite(X).all()

In [ ]:
X0, X1, y0, y1 = sk_selection.train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## baseline model

In [ ]:
base_model = sk_ensemble.HistGradientBoostingClassifier()

In [ ]:
base_model.fit(X0, y0)

In [ ]:
y_pred_base = base_model.predict(X1)

In [ ]:
print(sk_metrics.classification_report(y_true=y1, y_pred=y_pred_base))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.82      0.90      0.86        10
           2       0.89      0.80      0.84        10

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30

## neural net model

In [ ]:
ds_train = rnnm_data.NumpyTrainingDataset(X=X0, y=y0)
ds_valid = rnnm_data.NumpyTrainingDataset(X=X1, y=y1)

In [ ]:
ds_train[0]

In [ ]:
len(X0)

In [ ]:
from torch.utils.data import RandomSampler
import torch

In [ ]:
batch_size = 50

sampler = RandomSampler(
    ds_train,
    replacement=True,
    num_samples=int(2e3),
    generator=torch.manual_seed(42),
)

dl_train = DataLoader(
    ds_train,
    batch_size=batch_size,
    shuffle=False,
    sampler=sampler,
    collate_fn=rnnm_data.collate_numpy_dataset_to_xyblock_keep_orig_y,
    drop_last=True,
)
dl_valid = DataLoader(
    ds_valid,
    batch_size=batch_size,
    collate_fn=rnnm_data.collate_numpy_dataset_to_xyblock_keep_orig_y,
)

In [ ]:
next(iter(dl_train))

In [ ]:
n_hidden = [200, 100]
do_impute = False
impute_bias_source = rnnm_tab.BiasSources.zero
n_features = X0.shape[1]
n_classes = len(set(y0.ravel()))

model = rnnm_tab.TabularModelClassification(
    n_features=n_features,
    n_hidden=n_hidden,
    n_classes=n_classes,
    use_batch_norm=True,
)

In [ ]:
learning_rate = 0.1
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss = rnnm_loss.CrossEntropyXy()
loss_callback = rnnm_learner.TrainLossCallback()

save_dir = Path("./models")

callbacks = [loss_callback]

In [ ]:
learner = rnnm_learner.Learner(
    model,
    optimizer,
    loss,
    callbacks=callbacks,
    save_dir=save_dir,
    device=device,
    show_epoch_progress=True,
)

In [ ]:
lr_find_callback = rnnm_learner.LRFinderCallback(1e-5, 100, 100)

learner.find_learning_rate(
    dl_train, n_epochs=10, lr_find_callback=lr_find_callback
)

In [ ]:
# !mkdir models

In [ ]:
lr_find_callback.plot(yscale="log", ylim=(0.2, 5))

In [ ]:
learning_rate = 2e-3
n_epochs = 5

scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer=optimizer,
    max_lr=learning_rate,
    epochs=n_epochs,
    steps_per_epoch=len(dl_train),
)
scheduler_callback = rnnm_learner.EveryBatchSchedulerCallback(scheduler)
learner.update_callback(scheduler_callback)

In [ ]:
learner.fit(dl_train, n_epochs=n_epochs, dataloader_valid=dl_valid)

In [ ]:
loss_callback.plot(yscale="log")

In [ ]:
probs = learner.predict(dl_valid).detach().softmax(dim=1).numpy()
probs[:3]

In [ ]:
y_pred_nn = np.argmax(probs, axis=1)
y_pred_nn

In [ ]:
print(sk_metrics.classification_report(y_true=y1, y_pred=y_pred_nn))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.90      0.95        10
           2       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30